In [1]:
import pandas as pd
import numpy as np
from pyextremes import EVA
import matplotlib.pyplot as plt
import metpy.calc as mpcalc
from metpy.units import units

# Load data
file_path = "datos_viento.xlsx"
datos = pd.read_excel(file_path, sheet_name="Wind TdF ", usecols=[0, 1, 2])

In [2]:
# Intensidad
datos['Intensidad'] = np.sqrt(datos['u (m/s)']**2 + datos['v (m/s)']**2)

# Dirección
u = datos['u (m/s)'].values * units('m/s')
v = datos['v (m/s)'].values * units('m/s')
datos['Direccion'] = mpcalc.wind_direction(u, v, convention='from').m

def degToCompass(num):
    val = int((num / 45) + .5)
    arr = ["N", "NE", "E", "SE", "S", "SO", "O", "NO"]
    return arr[val % 8]

datos['Octante'] = datos['Direccion'].apply(degToCompass)


octantes = ["N", "NE", "E", "SE", "S", "SO", "O", "NO"]
ret_periods = [5, 50, 100]
resultados = pd.DataFrame(index=ret_periods, columns=octantes)

for octante in octantes:
    datos_octante = datos.copy()
    datos_octante.loc[datos_octante['Octante'] != octante, 'Intensidad'] = 0
    
    model = EVA(data=datos_octante.set_index('Fecha y Hora')['Intensidad'])
    model.get_extremes(method='BM', block_size='365D')
    model.fit_model()
    summary = model.get_summary(return_period=ret_periods, alpha=0.95)
    
    resultados[octante] = summary['return value']

print(resultados)

             N         NE          E         SE          S         SO  \
5    21.517956  19.386322  17.771155  14.339204  18.959229  24.996812   
50   23.005078  22.152813  28.516727  22.264626  19.908844  26.972172   
100  23.143136  22.629988  31.640319  24.568439  19.969535  27.201395   

             O         NO  
5    27.121692  25.074737  
50   30.481756  27.339622  
100  31.208910  27.678308  
